In [1]:
import numpy as np
import matplotlib.pyplot as plt

## matrix multiplication using quantization
### example 1
$arr1= \begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9 
\end{bmatrix}, arr2= \begin{bmatrix}
0.1 & 4 & 70 \\
0.2 & 5 & 80 \\
0.3 & 6 & 90 
\end{bmatrix} => dot(arr1,arr2)$

In [4]:
np.set_printoptions(precision=4, suppress=True)

arr1 = np.array([[1, 2, 3], [4,5,6], [7,8,9]], dtype=np.float32)
arr2 = np.array([[0.1, 0.2, 0.3], [4,5,6], [70, 80, 90]], dtype=np.float32).T

np.dot(arr1, arr2)

array([[   1.4,   32. ,  500. ],
       [   3.2,   77. , 1220. ],
       [   5. ,  122. , 1940. ]], dtype=float32)

## Step 1 Layerwise quantization (3bit)

### 1 array only 1 scale factor (and unsigned)
$ arr1 = (max(arr1) - min(arr1)) / (2^3-1) = 8/7 , \\ arr2 = (max(arr2)-min(arr2))/(2^3-1) = 89.9 / 7 $

In [14]:
nbits = 3
scale_arr1 = (arr1.max() - arr1.min()) / (2**nbits - 1)
quant_arr1 = np.round((arr1 - arr1.min())/scale_arr1)
dequant_arr1 = np.round((arr1 - arr1.min())/scale_arr1)*scale_arr1 + arr1.min()*scale_arr1
quant_error = np.linalg.norm(arr1 - dequant_arr1) # L2 norm

print("scale arr1 : ", scale_arr1)
print("quantize arr 1 : \n", quant_arr1)
print("dequantize arr 1 : \n", dequant_arr1)
print("quant error : ", quant_error)

scale_arr2 = (arr2.max() - arr2.min()) / (2**nbits - 1)
quant_arr2 = np.round((arr2 - arr2.min())/scale_arr2)
dequant_arr2 = np.round((arr2 - arr2.min())/scale_arr2)*scale_arr2 + arr2.min()*scale_arr2
quant_error2 = np.linalg.norm(arr2 - dequant_arr2) # L2 norm

print("scale arr2 : ", scale_arr2)
print("quantize arr 2 : \n", quant_arr2)
print("dequantize arr 2 : \n", dequant_arr2)
print("quant error 2: ", quant_error2)

scale arr1 :  1.1428571428571428
quantize arr 1 : 
 [[0. 1. 2.]
 [3. 3. 4.]
 [5. 6. 7.]]
dequantize arr 1 : 
 [[1.1429 2.2857 3.4286]
 [4.5714 4.5714 5.7143]
 [6.8571 8.     9.1429]]
quant error :  0.9583151
scale arr2 :  12.842857360839844
quantize arr 2 : 
 [[0. 0. 5.]
 [0. 0. 6.]
 [0. 0. 7.]]
dequantize arr 2 : 
 [[ 1.2843  1.2843 65.4986]
 [ 1.2843  1.2843 78.3414]
 [ 1.2843  1.2843 91.1843]]
quant error 2:  8.448837


## Step 2 row-wise quantization (3bit)

### matrix multiplication
$ C_{ij} = \sum $ \
$ arr1 = (max(arr1) - min(arr1)) / (2^3-1) = 8/7 , \\ arr2 = (max(arr2)-min(arr2))/(2^3-1) = 89.9 / 7 $

## Step 2 row-wise quantization (3bit)

### matrix multiplication
$ C_{ik} = \sum^{k}arr1_{ij}*arr2_{jk} $ \
arr1 is row-wise independent ($arr1_{1j}$ does not interfere $arr1_{2j}$) \
arr2 is column-wise independent ($arr2_{j1}$ does not interfere $arr2_{j2}$) \
$ arr1_{0k} = (max(arr1_{0k}) - min(arr1_{0k})) / (2^3-1) = 2/7 , \\ arr2_{k0} = (max(arr2_{k0})-min(arr2_{k0}))/(2^3-1) = 0.3-0.1 / 7 $